In [1]:
# run up the shell with  

# pyspark --packages graphframes:graphframes:0.5.0-spark2.1-s_2.11

# to allow for the graphframes import


In [98]:
from graphframes import GraphFrame
from pyspark.sql.functions import col

In [99]:
v = sqlContext.createDataFrame([
  ("CH1", "CH"),
  ("CH2", "CH"),
  ("CH3", "CH"),
  ("VAT1", "VAT"),
  ("VAT2", "VAT"),
  ("VAT3", "VAT"),
  ("PAYE1", "PAYE"),
  ("PAYE2", "PAYE"),
  ("PAYE3", "PAYE"),
], ["id", "type"])

In [100]:
e = sqlContext.createDataFrame([
  ("CH1", "VAT1", "0.8"),
  ("CH1", "VAT2", "0.9"),
  ("CH2", "VAT2", "0.91"),
  ("VAT1", "PAYE1", "0.86"),
], ["src", "dst", 'weight'])

In [101]:
# Create a GraphFrame
g = GraphFrame(v, e)

In [102]:
sc.setCheckpointDir(dirName='/Users/waltoj/development/datascience/graph-frames-prototype/checkpoints')

In [103]:
connected_components = g.connectedComponents()

In [104]:
#connected_components.collect()

In [105]:
# component_ids = connected_components.select('component').distinct()
component_ids = connected_components.select('component').distinct().rdd.map(lambda r: r[0]).collect()

In [106]:
component_ids

[1443109011456, 1314259992576, 231928233984, 1357209665536, 661424963584]

In [107]:
def get_subgraph_vertices(connected_components, component_id):
    return connected_components.where(col('component') == component_id)

In [108]:
def get_sub_graph_edge_list(vertex_ids, edge_list):
    """
    returns empty list if it's a single, as there are no edges
    """
    return edge_list.where(col('src').isin(vertex_ids.collect()) | col('dst').isin(vertex_ids.collect()))

In [109]:
def get_subgraph_components(component_id, connected_components, edge_list):
    """
    Return edge list and vertex lists of the connected component generated from the connected_components algorithm
    """
    subgraph_vertex_list = get_subgraph_vertices(connected_components, component_id) 
    subgraph_vertices_ids = subgraph_vertex_list.distinct().rdd.map(lambda r: r[0])
    
    sub_graph_edge_list = get_sub_graph_edge_list(subgraph_vertices_ids , edge_list)
    
    return subgraph_vertex_list, sub_graph_edge_list

In [110]:
def multiple_ch_nodes(vertex_list):  
    number_of_ch_nodes = vertex_list.where(vertex_list['type'] == 'CH').count()
    return number_of_ch_nodes >= 2

In [115]:
def split_complex_sub_graphs(vertex_list, edge_list):
    G = GraphFrame(vertex_list, edge_list)
    # need the locations of the CH nodes for the 
    ch_nodes = vertex_list.where(vertex_list['type'] == 'CH')
    shortest_paths = G.shortestPaths(ch_nodes.rdd.map(lambda r: r[0]).collect())
    print(shortest_paths.collect())
    return

In [116]:
def process_subgraph(vertex_list, edge_list):
    """
    Check generated legal units are valid and decompose any which aren't into smaller legal units
    param: dataframe of the edge list of the sub graph
    """
    
    #TODO: check if 2 CH's, split graph if t
    if multiple_ch_nodes(vertex_list):
        return split_complex_sub_graphs(vertex_list, edge_list)
    # return a list of the subgraphs (list of lists)
    # [[CH!, VAT2], [CH2, VAT3]]
    
    return
     

In [117]:
# create output dataframe to append too

In [ ]:
for component_id in component_ids:
    subgraph_vertex_list, subgraph_edge_list = get_subgraph_components(component_id, connected_components, e)
    #print(subgraph_vertex_list.collect())
    #print(subgraph_edge_list.collect())

    sub_graphs = process_subgraph(subgraph_vertex_list, subgraph_edge_list)
    # add final_sub_graph to an output df, what about singles which aren't picked up from get_subgraph?
    